# Немного простого кодирования

Для того чтобы размяться, расслабиться и получить ещё немножко удовольствия и очков.

## A. Алгоритм Евклида

[Тот самый](http://e-maxx.ru/algo/export_euclid_algorithm).

Даны $a$ и $b$.

$$
\text{gcd}(a,b) = \begin{cases}
a, & b = 0\\
\text{gcd}(b, a\mod{}b), & b \ne 0
\end{cases}
$$

Реализовать $\text{gcd}$ на Python.

In [32]:
def gcd(a : 'int', b : 'int') -> 'int':
    """Return the greatest common divisor of a and b"""
    a, b = abs(a), abs(b)
    while b:
        a,b = b, a % b
    return a

print(gcd(91, 21))

7


## B. Реализовать функцию, которая отвечает, простое число, или нет

Т.е. возвращает `True`, если оно простое, и `False` в противном случае.
Можно совершенно «в лоб», с проверкой на делимость, но чтобы проверок было не больше, чем необходимо.

Но поскольку это решение неоптимально, следует воспользоваться возможностями *мемоизации* — декоратором [`@functools.lru_cache`](https://docs.python.org/3/library/functools.html#functools.lru_cache). Он похволяет кешировать результаты функций **без побочных эффектов**, чтобы не считать многократно то, что уже посчитано. При этом при помощи директивы `%timeit` можно посмотреть, насколько мемоизация ускоряет работу функции.

In [7]:
import functools
import math
import random


def gcd(a : 'int', b : 'int') -> 'int':
    """Return the greatest common divisor of a and b"""
    a, b = abs(a), abs(b)
    while b:
        a,b = b, a % b
    return a
    
    
def miller_rabin(n : 'int', base : 'int') -> 'bool':
    """Perform a single iteration of a Miller-Rabin primality test"""
    s = 0
    t = n - 1
    while t % 2 == 0:
        s += 1
        t //= 2
    # n - 1 = 2^s * t
    x = pow(base, t, n)
    if x == 1 or x == n-1:
        return True
    for i in range(s):
        x = pow(x, 2, n)
        if x == 1:
            return False
        if x == n-1:
            return True
    return False

def perfect_square(n):
    x = n
    y = (x + 1) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x * x == n

def jacobi(d : 'int', n : 'int') -> 'int':
    """Calculate the Jacobi symbol with numerator d and denumerator n"""
    t = gcd(d, n)
    if t != 1:
        return 0
    sign = 1
    if (d == 1 or n == 1):
        return 1
    while True:
        d = d % n
        while d % 2 == 0:
            d //= 2
            n8 = n % 8
            if n8 == 3 or n8 == 5:
                sign *= -1
        if d == 1:
            return sign
        if d < n:
            if n % 4 == 3 and d % 4 == 3:
                sign *= -1
            d, n = n, d
        

    
    
def lucas_selfridge_test(n : 'int') -> 'bool':
    D = 5
    D_sign = 1
    while jacobi(D * D_sign, n) != -1:
        D += 2
        D_sign *= -1
    D = D * D_sign
    P = 1
    Q = (1 - D) // 4
    u, v, u2, v2, q, q2, qd = 0, 2, 1, P, Q, 2 * Q, Q
    bits = []
    t = (n + 1) // 2
    while t > 0:
        bits.append(t % 2)
        t = t // 2
    h = 0
    while h < len(bits):
        u2 = (u2 * v2) % n
        v2 = (v2 * v2 - q2) % n
        if bits[h] == 1:
            uold = u
            u = u2 * v + u * v2
            u = u if u % 2 == 0 else u + n
            u = (u // 2) % n
            v = (v2 * v) + (u2 * uold * D)
            v = v if v % 2 == 0 else v + n
            v = (v // 2) % n
        if h < len(bits) - 1:
            q = (q * q) % n
            q2 = q + q
        h = h + 1
    if u == 0:
        return True
    return False



def is_simple(value: 'int') -> 'bool':
    """Bailie–PSW probabilistic primality test"""
    if (value == 1):
        return False
    # Trivial test used to speed up computation when a number has
    # a small prime factor.
    primes = (2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
          47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97)
    if value in primes:
        return True
    for i in primes:
        if value % i == 0:
            return False
    # A single iteration of Miller-Rabin test with base 2
    if not miller_rabin(value, 2):
        return False
    # Check if the number is a perfect square, this is
    # necessary for the next test
    if perfect_square(value):
        return False
    # Perform Lucas pseudoprime test with Selfridge parameters
    return lucas_selfridge_test(value)

def is_simple_random(value):
    if (value == 1):
        return False
    primes = (2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
          47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97)
    if value in primes:
        return True
    for i in primes:
        if value % i == 0:
            return False
    for i in range(10):
        base = random.randrange(2, value - 2)
        if not miller_rabin(value, base):
            return False
    return True

is_simple_cached = functools.lru_cache()(is_simple)

prime1 = 2 ** 521 - 1
prime2 = 2 ** 1279 - 1
prime3 = 2 ** 2203 - 1
prime4 = 2 ** 3217 - 1
prime5 = 2 ** 4253 - 1

print(is_simple(1))
print(is_simple(2))
print(is_simple(19))
print(is_simple(91))
print(is_simple(prime1))
print(is_simple(prime2))
print(is_simple(prime3))
print(is_simple(prime4))
print(is_simple(prime1 * prime2))

%timeit is_simple(prime5)


%timeit is_simple(prime4)
%timeit is_simple_random(prime4)
%timeit is_simple_cached(prime4)

%timeit is_simple(prime1 * prime2)
%timeit is_simple_random(prime1 * prime2)
%timeit is_simple_cached(prime1 * prime2)



False
True
True
False
True
True
True
True
False
567 ms ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
324 ms ± 3.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.16 s ± 34.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.35 µs ± 818 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)
19.3 ms ± 409 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
22.1 ms ± 434 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.21 µs ± 17.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## C. Вывод чисел в системе счисления с произвольным (но не более 36) основанием

Если основание не превосходит $B \le 10$, используются цифры $\{0,1,\ldots, B-1\}$. Если от $10 < B \le 36$, то $\{0,\ldots, 10, A \ldots\}$ в количестве $B$.

Конструктор `int` в Python умеет получать $B$ на вход, что много кого не раз выручало:

In [ ]:

def str_base(value: 'int', base: 'int') -> 'str':
    """Shows value using number base, e.g. str_base(44027, 36)=='XYZ'"""
    if value == 0:
        return '0'
    s = ''
    while value > 0:
        num = value % base
        if num < 10:
            s = str(num) + s
        else:
            s = chr(ord('A') + num - 10) + s
        value = value // base
    return s
        
    

print(str_base(44027, 36))  # хочется увидеть 'XYZ'

## D. Расширенный Алгоритм Евклида

Тоже [тот самый](http://e-maxx.ru/algo/export_extended_euclid_algorithm).

Даны $a$ и $b$. Обратиться к своим знаниям алгебры (если они не помогли, то можно сходить по ссылке выше) и реализовать $\text{egcd}$, такую что $\text{egcd}(a, b) = \left<x, y, \text{gcd}(a, b)\right>$, такие что $\text{gcd}(a, b) = ax + by$.

In [4]:

def egcd(a, b):
    sign = [1 if x > 0 else -1 for x in (a, b)]
    a, b = abs(a), abs(b)
    rem_a = [a]
    while b:
        a,b = b, a % b
        rem_a.append(a)
    c, d = 1, 0
    n = len(rem_a)
    for m,n in zip(rem_a[-2::-1], rem_a[::-1]):
        c, d = d, c - m // n * d
    return (c*sign[0], d*sign[1], a)

def egcd_recursive(a, b):
    if b == 0:
        return (1, 0, a)
    x, y, g = egcd_recursive(b, a % b)
    return (y, x - a // b * y, g)
0
def fib(n):
    a = 0
    b = 1
    for i in range(n):
        a, b = b, a + b
    return a

fib1 = fib(3000)
fib2 = fib(3001)

try:
    print(egcd_recursive(fib2, fib1))
except RecursionError:
    print("oof ouch owie")


n = 91
m = 21
t = egcd(n, m)
t2 = egcd_recursive(n, m)
print(t, t2)
print(n*t[0] + m*t[1])

oof ouch owie
(1, -4, 7) (1, -4, 7)
7
